# Topo solvers

In this tutorial you learn how to
- use optimization criteria as objective functions or as evaluation metrics
- use our topo solvers, particularly our SIMP topo solver

Finally, we start actually performing topology optimization! 

The TopoSolver class is a parent class that inherits all types of methods for solving TO problems. This contains the famous SIMP method, DL based methods, as well as arbitrary combinations of these two.

In this chapter we start with an introduction of how our optimization pipeline works. For now, we will focus mainly on the SIMP method an we will discuss learned methods in the next tutorial.

### Criteria

For any topo solver we need to choose an optimization criterion, i.e., an objective function that for which we want to optimize. 

For example, let's say we want to minimize the volume of a structure. Depending on what our exact goal is, there are three possible criteria for that:
1. The volume criterion. This criterion simply sums up all values in the density distribution.
2. The volume fraction criterion. This criterion returns the volume of the structure divided by the total number of voxels, i.e. the average density. This has the advantage that it always returns a value between $0$ and $1$.
3. The volume constraint criterion. This criterion compares the volume fraction wit a pre-defined maximum volume fraction. If the difference is negative (i.e., the current volume is below the maximum volume fraction) then this criterion returns $0$, otherwise it returns a positive output. The thresholding is done by either a ReLU function, or a Softplus function for more smoothness.

Our library contains many more criteria that can be used for evaluation or as loss/objective functions. We will refer to some of them in the next tutorial on trainable topo solvers. For now, we will use the volume fraction and the stress constraint criterion for SIMP. The stress constraint criterion has a similar concept like the volume constraint criterion: We compare the current maximal von Mises stress with the yield stress and we get an output $>0$ if the von Mises stress is larger than the yield stress.

We weight the StressConstraint accordingly. Unfortunately, the SIMP algorithm - like most algorithms - needs some sort of finetuning that is done manually, usually by trial-end-error. We found that a constant of $10^{-11}$ works well for this example.

In [ ]:
#ignore
from dl4to.criteria import VolumeFraction, StressConstraint

criterion = VolumeFraction() + 1e-11 * StressConstraint(threshold_fct='softplus')

For our TO problem we choose the ledge problem introduced in the last tutorials, and we pass an FDM solver to it:

In [ ]:
#ignore
from dl4to.datasets import BasicDataset
from dl4to.pde import FDM

problem = BasicDataset(resolution=40).ledge(force_per_area=-1e6)
problem.pde_solver = FDM()

### The SIMP method

The Solid Isotropic Material with Penalization (SIMP) method is widely regarded as the most significant classical approach used in TO. SIMP employs an iterative optimization scheme to improve structural performance by adjusting voxel densities. See Algorithm 1 for a full breakdown of the steps that are part of the SIMP algorithm:

![title](https://dl4to.github.io/dl4to/images/3_simp.png)

For now, we don't really need to know how the algorithm works exactly. We can initialize SIMP as follows:

In [ ]:
#ignore
from dl4to.topo_solvers import SIMP

simp = SIMP(
    criterion=criterion,
    n_iterations=100,
    binarizer_steepening_factor=1.06,
    lr=1e-1,
)

### How to apply topo solvers to a TO problem

In order to apply a topo_solver to a problem, we can simply call it via

    solution = topo_solver(problem),

which returns a solution object. This also works with a list of problems as input, in which case topo solver likewise returns a list of solutions.

In [ ]:
#ignore
solution = simp(problem)

  0%|          | 0/100 [00:00<?, ?it/s]

Let's take a look at the solution of the SIMP algorithm:

In [ ]:
#ignore
camera_position = (0, 0.35, 0.2)
solution.plot(camera_position=camera_position,
              solve_pde=True,
              display=False)

![density](https://dl4to.github.io/dl4to/images/3_density.png)
![displacements](https://dl4to.github.io/dl4to/images/3_displacements.png)
![stresses](https://dl4to.github.io/dl4to/images/3_stresses.png)

We see that the output of the SIMP algorithm returns indeed a plausible solution to the TO problem.

We can check the volume fraction of this solution via the following:

In [ ]:
#ignore
volume_fraction = VolumeFraction()
solution.eval(volume_fraction)

tensor([0.4208], grad_fn=<DivBackward0>)

We also implemented a "Binariness" criterion which determines how binary the solution is. A value of $1$ means that it is fully binary, while a lower value signifies many values that are not near $0$ or $1$.

In [ ]:
#ignore
from dl4to.criteria import Binariness

binariness = Binariness()
solution.eval(binariness)

tensor([0.9989])

The binariness score is very high, which confirms the impression that we get from the 3d plot above.